1. **Nome(s): Jordano Furtado de Souza, Felipe Sala Carpes, Marcos Vinícius Silva**
2. **Email : jordano.souza@edu.ufes.br, felipe.carpes@edu.ufes.br, marcos.s.silva@edu.ufes.br**
3. **Matrícula(s): 2022101183, 2022102425, 2020100954**
4. **Curso(s): Engenharia de Computação**

# Projeto Final - Ciência de Dados

### Metodologia
  - Introdução
  - Definição do problema
  - Descrição do conjunto de dados
  - Metodologia
  - Resultados
  - Conclusões e discussões


## 1. Processamento da Tabela de Pedidos (`orders_dataset`)

O primeiro passo do pipeline consiste em transformar as datas brutas de logística em métricas de desempenho operacional. O objetivo é converter instantes temporais em **intervalos de tempo (Timedeltas)**, que possuem maior poder preditivo para o nosso classificador de satisfação.

As novas features criadas são:
1. **Tempo de Entrega Real**: O período total que o cliente esperou pelo produto.
2. **Tempo de Delay (Atraso)**: A diferença entre a promessa de entrega e a realidade (valores positivos indicam atraso).
3. **Tempo de Despache**: Eficiência do vendedor e da logística interna até a entrega para a transportadora.

Após o cálculo, realizamos a limpeza do dataframe para manter apenas as métricas de interesse e a referência temporal.

In [36]:
import pandas as pd

# carregar dataset
df_raw = pd.read_csv('dataset/raw/orders_dataset.csv')

# remover pedidos que ainda não foram entregues
df_processed_time = df_raw[df_raw['order_status'] == 'delivered'].copy()

In [54]:
cols_date = [
    'order_purchase_timestamp', 
    'order_approved_at', 
    'order_delivered_carrier_date', 
    'order_delivered_customer_date', 
    'order_estimated_delivery_date'
]
for col in cols_date:
    df_processed_time[col] = pd.to_datetime(df_processed_time[col])

In [40]:
# 2. Criação das novas colunas (calculadas em dias)
# Usamos .dt.total_seconds() / 86400 para ter o valor numérico em dias
df_processed_time['tempo_entrega_real'] = (df_processed_time['order_delivered_customer_date'] - df_processed_time['order_purchase_timestamp']).dt.total_seconds() / 86400
df_processed_time['tempo_delay'] = (df_processed_time['order_delivered_customer_date'] - df_processed_time['order_estimated_delivery_date']).dt.total_seconds() / 86400
df_processed_time['tempo_despache'] = (df_processed_time['order_delivered_carrier_date'] - df_processed_time['order_approved_at']).dt.total_seconds() / 86400
df_processed_time[['tempo_entrega_real', 'tempo_delay', 'tempo_despache']].head()

,tempo_entrega_real,tempo_delay,tempo_despache
0,8.436574,-7.107488,2.366493
1,13.782037,-5.355729,0.462882
2,9.394213,-17.245498,0.204595
3,13.208750,-12.980069,3.745833
4,2.873877,-9.238171,0.893113


In [57]:
# 3. Filtragem das colunas finais
colunas_finais = [
    'order_id', 
    'customer_id', 
    'order_purchase_timestamp',  # Mantendo a referência temporal
    'tempo_entrega_real', 
    'tempo_delay', 
    'tempo_despache'
]
# salvando tabela
df_final_time = df_processed_time[colunas_finais].copy()

# Visualizar o resultado
df_final_time.head()

,order_id,customer_id,order_purchase_timestamp,tempo_entrega_real,tempo_delay,tempo_despache
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,2017-10-02 10:56:33,8.436574,-7.107488,2.366493
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,2018-07-24 20:41:37,13.782037,-5.355729,0.462882
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,2018-08-08 08:38:49,9.394213,-17.245498,0.204595
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,2017-11-18 19:28:06,13.208750,-12.980069,3.745833
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,2018-02-13 21:18:39,2.873877,-9.238171,0.893113


## 2. Processamento da Tabela de Itens (`order_items_dataset`)

Nesta etapa, analisamos a "dor financeira" do cliente. A percepção de valor e o custo do frete são fatores psicológicos determinantes na tolerância a falhas logísticas. 

Para isso, realizamos uma **agregação por pedido**, calculando o valor total dos itens e do frete. Introduzimos também uma métrica de **proporcionalidade**, que indica o quão "caro" o frete pareceu para o consumidor em relação ao produto adquirido.

As métricas geradas são:
1. **Valor Total do Pedido**: Soma dos preços de todos os itens do pedido.
2. **Valor Total do Frete**: Custos logísticos totais acumulados.
3. **Quantidade de Itens**: Número total de produtos no carrinho (indicador de complexidade logística).
4. **Ratio Frete/Produto**: Razão entre frete e valor dos itens (Métrica de percepção de custo-benefício).
5. **Ticket Médio por Item**: Pedidos com itens caros tendem a gerar maior ansiedade no cliente

In [87]:
# carregar dataset
df_raw = pd.read_csv('dataset/raw/order_items_dataset.csv')
df_processed_price = df_raw.copy()
df_processed_price.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [89]:
# agregação
df_processed_price = df_processed_price.groupby('order_id').agg({
    'price': 'sum',                # valor total de todos os itens
    'freight_value': 'sum',        # valor total do frete
    'order_item_id': 'count',      # quantidade de itens
}).reset_index()

In [88]:
# ajustando nome colunas
df_processed_price.columns = ['order_id', 'product_id', 'vl_total_pedido', 'vl_frete_total', 'qtd_itens']

# nova feature de proporcionalidade, entre o valor do pedido com o frete 
df_processed_price['ratio_frete_produto'] = df_processed_price['vl_frete_total'] / df_processed_price['vl_total_pedido']

# nova feature de preço médio do produto, identifica se foi uma compra de produtos caros ou muitos produtos
df_processed_price['ticket_medio_item'] = df_processed_price['vl_total_pedido'] / df_processed_price['qtd_itens']

ValueError: Length mismatch: Expected axis has 7 elements, new values have 5 elements

In [58]:
# salvando tabela
df_final_price = df_processed_price.copy()

# Visualizar o resultado
df_final_price.head()

,order_id,price,freight_value,order_item_id
0,00010242fe8c5a6d1ba2dd792cb16214,58.90,13.29,1
1,00018f77f2f0320c557190d7a144bdd3,239.90,19.93,1
2,000229ec398224ef6ca0657da4fc703e,199.00,17.87,1
3,00024acbcdf0a6daa1e931b038114c75,12.99,12.79,1
4,00042b26cf59d7ce69dfabb4e55b4fd9,199.90,18.14,1


## 3. Processamento da Tabela de Produtos (products_dataset)

O objetivo desta etapa é extrair características físicas e de categoria que possam influenciar a expectativa do cliente. Produtos "complexos" (como eletrônicos com descrições longas) ou produtos "críticos" (grandes e pesados) tendem a ter um comportamento de avaliação diferente de itens pequenos e simples.

As métricas geradas são:
1. **Volume do Produto**: Calculado a partir das dimensões (comprimento × altura × largura).
2. **Densidade do Produto**: Relação entre peso e volume, útil para identificar produtos frágeis ou muito robustos.
3. **Nível de Detalhamento**: Comprimento da descrição e nome do produto (hipótese: descrições muito curtas podem gerar expectativas erradas).

In [83]:
import numpy as np
# carregar dataset
df_raw = pd.read_csv('dataset/raw/products_dataset.csv')
df_processed_products = df_raw.copy()

df_processed_products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0


In [82]:
# calculando o volume em cm³
df_processed_products['product_volume_cm3'] = (
    df_processed_products['product_length_cm'] * df_processed_products['product_height_cm'] * df_processed_products['product_width_cm']
)

# ajuste pra nao ocorrer divisão pro zero
df_processed_products['product_volume_cm3'] = df_processed_products['product_volume_cm3'].replace(0, np.nan)

# calculando a densidade 
df_processed_products['product_density'] = df_processed_products['product_weight_g'] / (df_processed_products['product_volume_cm3'])

In [55]:
# tratando de possíveis informações faltantes
df_processed_products['product_description_lenght'] = df_processed_products['product_description_lenght'].fillna(0)
df_processed_products['product_name_lenght'] = df_processed_products['product_name_lenght'].fillna(0)

df_processed_products.head()

,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_volume_cm3,product_density
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,40.0,287.0,1.0,225.0,16.0,10.0,14.0,2240.0,0.100446
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,44.0,276.0,1.0,1000.0,30.0,18.0,20.0,10800.0,0.092593
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,46.0,250.0,1.0,154.0,18.0,9.0,15.0,2430.0,0.063374
3,cef67bcfe19066a932b7673e239eb23d,bebes,27.0,261.0,1.0,371.0,26.0,4.0,26.0,2704.0,0.137204
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,37.0,402.0,4.0,625.0,20.0,17.0,13.0,4420.0,0.141403


In [60]:
cols_products = [
    'product_id',
    'product_category_name',
    'product_volume_cm3',
    'product_density',
    'product_description_lenght',
    'product_photos_qty'
]
# salvando tabela, só as colunas importantes
df_final_products = df_processed_products[cols_products].copy()

# Visualizar o resultado
df_final_products.head()

,product_id,product_category_name,product_volume_cm3,product_density,product_description_lenght,product_photos_qty
0,1e9e8ef04dbcff4541ed26657ea517e5,perfumaria,2240.0,0.100446,287.0,1.0
1,3aa071139cb16b67ca9e5dea641aaa2f,artes,10800.0,0.092593,276.0,1.0
2,96bd76ec8810374ed1b65e291975717f,esporte_lazer,2430.0,0.063374,250.0,1.0
3,cef67bcfe19066a932b7673e239eb23d,bebes,2704.0,0.137204,261.0,1.0
4,9dc1a7de274444849c219cff195d0b71,utilidades_domesticas,4420.0,0.141403,402.0,4.0


## 4. Processamento da Tabela de Clientes (customers_dataset)

O objetivo desta etapa é extrair a localização do consumidor. 

In [63]:
# carregar o dataset
df_raw = pd.read_csv('dataset/raw/customers_dataset.csv')

cols_customers = [
    'customer_id',
    'customer_zip_code_prefix',
    'customer_city',
    'customer_state'
]

In [65]:
df_processed_costumers = df_raw[cols_customers].copy()

# padronizar a escrita
df_processed_costumers['customer_city'] = df_processed_costumers['customer_city'].str.upper()
df_processed_costumers['customer_state'] = df_processed_costumers['customer_state'].str.upper()

# salvando tabela, só as colunas importantes
df_final_costumers = df_processed_costumers.copy()

# Visualizar o resultado
df_final_costumers.head()

,customer_id,customer_zip_code_prefix,customer_city,customer_state
0,06b8999e2fba1a1fbc88172c00ba8bc7,14409,FRANCA,SP
1,18955e83d337fd6b2def6b18a428ac77,9790,SAO BERNARDO DO CAMPO,SP
2,4e7b3e00288586ebd08712fdd0374a03,1151,SAO PAULO,SP
3,b2b6027bc5c5109e529d4dc6358b12c3,8775,MOGI DAS CRUZES,SP
4,4f2d8ab171c80ec8364f7c12e35b23ad,13056,CAMPINAS,SP


## 5. Processamento da Tabela de Avaliações (order_reviews_dataset)

O objetivo do nosso trabalho.

- Classe 0 (Insatisfeito): Notas 1, 2 e 3.
- Classe 1 (Satisfeito): Notas 4 e 5.

In [72]:
# carregar o dataset
df_raw = pd.read_csv('dataset/raw/order_reviews_dataset.csv')
df_processed_reviews = df_raw.copy()


In [73]:
def binarizar_nota(score):
    if score >= 4:
        return 1 # Satisfeito
    else:
        return 0 # Insatisfeito

In [79]:
df_processed_reviews['target_label'] = df_processed_reviews['review_score'].apply(binarizar_nota)
df_final_reviews = df_processed_reviews[['order_id', 'review_score', 'target_label']].copy()
df_final_reviews.head()

,order_id,review_score,target_label
0,73fc7af87114b39712e6da79b0a377eb,4,1
1,a548910a1c6147796b98fdf73dbeba33,5,1
2,f9e4b658b201a9f2ecdecbb34bed034b,5,1
3,658677c97b385a9be170737859d3511b,5,1
4,8e6bfb81e283fa7e4f11123a3fb894f1,5,1


## 6. Construção Tabela Final 
Unificamos todas as informações construídas anteriormente (Logística, Financeira, Produto e Cliente) em um único DataFrame. 

**Chaves de Ligação:**
- `order_id`: Para unir Logística, Preços e Produtos.
- `customer_id`: Para unir a localização do Cliente (já que esta informação não está na tabela de reviews).